In [3]:
import pandas as pd
import re
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments,pipeline
import torch
from datasets import load_dataset

Обучение модели самостоятельно

In [28]:
#импорт датасета
data = pd.read_csv('jokes.csv')

print(data.head())

       theme                                               text  rating
0  pro-sudey  На суде в Стамбуле обвиняемый сказал:\r\n- На...       5
1  pro-sudey  - Вы продолжаете утверждать, что обвиняемый н...       4
2  pro-sudey  На суде.\r\n- Итак, когда дело дошло до столкн...       0
3  pro-sudey  Старую леди сбил автомобиль. На суде ее спраши...       4
4  pro-sudey  Судья говорит:\r\n- Согласно вашей жалобе, об...       2


In [29]:
#Изучение датасета

# Посмотрим на количество строк и столбцов
print(f"Размер датасета: {data.shape}")

# Выведем информацию о датасете
print(data.info())

# Проверим наличие пропущенных значений
print(data.isnull().sum())

# Посмотрим на уникальные темы
print(data['theme'].unique())


Размер датасета: (130204, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130204 entries, 0 to 130203
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   theme   130204 non-null  object
 1   text    130204 non-null  object
 2   rating  130204 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.0+ MB
None
theme     0
text      0
rating    0
dtype: int64
['pro-sudey' 'pro-studentov' 'pro-sport-pro-futbol' 'pro-sisadminov'
 'pro-semyu' 'pro-poruchika-rgevskogo' 'pro-putina' 'pro-programmistov'
 'pro-novih-russkih' 'pro-narkomanov' 'pro-mugchin' 'pro-militsiyu'
 'pro-kompyuteri' 'pro-kino' 'pro-inostrantsev' 'pro-givotnih'
 'pro-genshin' 'pro-evreev' 'pro-druzey' 'pro-detey' 'pro-vovochku'
 'pro-buhgalterov' 'pro-billa-geytsa' 'pro-armiu' 'pro-alkogolikov'
 'pro-wow' 'poshlie-i-intimnie' 'politicheskie' 'narodnie' 'meditsinskie'
 'kriminalnie' 'cherniy-yumor' 'tsitati' 'sovetskie' 'skazochnie' 'raznie'
 'pro-shtirlits

In [30]:
#Очистка данных
# Удаляем строки с пропущенными значениями
data = data.dropna(subset=['text'])

# Удаляем дубликаты
data = data.drop_duplicates(subset=['text'])

# Сбрасываем индексы
data = data.reset_index(drop=True)
print(f"Размер датасета после очистки: {data.shape}")

Размер датасета после очистки: (129383, 3)


In [31]:
#Предобработка текста
def preprocess_text(text):
    # Удаляем лишние пробелы
    text = ' '.join(text.split())
    # Удаляем нежелательные символы (если есть)
    text = re.sub(r'[^\w\s.,!?«»—-]', '', text)
    return text.strip()

# Применяем предобработку к столбцу 'text'
data['text'] = data['text'].apply(preprocess_text)

In [32]:
#Объединение текстов в один корпус
# Определяем специальный токен (используется в GPT-2)
eos_token = '<|endoftext|>'

# Объединяем все анекдоты в один текст
corpus = eos_token.join(data['text'].tolist())

In [33]:
# Сохранение анекдотов с разделителем, каждый анекдот на новой строке
with open('jokes_corpus.txt', 'w', encoding='utf-8') as f:
    for joke in data['text']:
        f.write(joke + ' <|endoftext|>\n')

Выбор размера модели

In [9]:
small_model = 'sberbank-ai/rugpt3small_based_on_gpt2'

In [34]:
medium_model = 'sberbank-ai/rugpt3medium_based_on_gpt2'

In [35]:
tokenizer = GPT2Tokenizer.from_pretrained(medium_model)
model = GPT2LMHeadModel.from_pretrained(medium_model)

In [36]:
# Создаем датасет из текстового файла
dataset = load_dataset('text', data_files='jokes_corpus.txt')

Generating train split: 0 examples [00:00, ? examples/s]

In [37]:
# Разделяем датасет на обучающую и проверочную выборки
train_testsplit = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_testsplit['train']
eval_dataset = train_testsplit['test']

In [38]:
print(f"Обучающая выборка: {len(train_dataset)} образцов")
print(f"Проверочная выборка: {len(eval_dataset)} образцов")

Обучающая выборка: 116444 образцов
Проверочная выборка: 12939 образцов


In [39]:
checkpoint_path = './results/checkpoint-145555' #Продолжение обучения с чекпоинта

In [40]:
#Третий раз
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=7,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="steps",  
    eval_steps=5000,
    save_strategy="steps",
    save_steps=5000,
    logging_steps=1000,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    resume_from_checkpoint=checkpoint_path,
)

In [41]:
#Токенизация датасета
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/116444 [00:00<?, ? examples/s]

Map:   0%|          | 0/12939 [00:00<?, ? examples/s]

In [42]:
#Установка формата данных
tokenized_train_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])
tokenized_eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])

In [43]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [44]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator,
)

In [20]:
#Обучение модели
trainer.train()

Step,Training Loss,Validation Loss
5000,2.938100,2.843206
10000,2.861000,2.778905
15000,2.820400,2.739632
20000,2.786200,2.725641
25000,2.777900,2.701463
30000,2.397700,2.704596
35000,2.395800,2.693990
40000,2.386300,2.696573
45000,2.408600,2.681817
50000,2.403000,2.675772


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=203777, training_loss=1.8872390088352364, metrics={'train_runtime': 74110.7992, 'train_samples_per_second': 10.999, 'train_steps_per_second': 2.75, 'total_flos': 1.8924784142883226e+17, 'train_loss': 1.8872390088352364, 'epoch': 7.0})

In [21]:
#Сохранение модели
trainer.save_model('./models/trained_model')
tokenizer.save_pretrained('./models/trained_model')

('./models/trained_model/tokenizer_config.json',
 './models/trained_model/special_tokens_map.json',
 './models/trained_model/vocab.json',
 './models/trained_model/merges.txt',
 './models/trained_model/added_tokens.json')

In [4]:
# Загрузка дообученной модели из сохранённой директории
model_path = './models/trained_model' 
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

In [13]:
prompt = "Заходит англичанин в бар"

In [11]:
prompt = "Попали на необитаемый остров немец, американец и русский"

In [5]:
prompt = "Приходит вовочка на урок математики"

In [14]:
model.to('cuda')

generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

generated_joke = generator(
    prompt,
    max_length=250,
    truncation=True,
    num_return_sequences=1,
    no_repeat_ngram_size=3,
    repetition_penalty=1.2,
    top_p=0.8,
    temperature=0.6,
    do_sample=True,
    top_k=40,
    eos_token_id=tokenizer.eos_token_id,
)



joke = generated_joke[0]['generated_text']
joke = joke.split('<|endoftext|>')[0].strip()
print(joke)

Заходит англичанин в бар. - Мне виски, пожалуиста. Бармен наливает ему виски и спрашивает - А как вас зовут? Англичанин отвечает - Я Робин Гуд. Бармена это очень устраивает, он выпивает еще несколько стаканов и просит повторить. Через некоторое время опять заходит англичанин. - Я Винни Пух. Бармены его спрашивают - Ну что ж вы так, Винни... - Да нет же! Меня зовут Робин Гуд, а меня никто не знает, я никому не скажу, потому что у меня имя такое странное - Брутманн!!!...


Воспользуемся обученной моделью

In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import pipeline

In [2]:
# Загрузка модели и токенизатора
model_name = 'gpt2'  # Можно использовать 'gpt2-medium', 'gpt2-large', 'gpt2-xl' для более мощных моделей
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/home/vadim/inopolis_project/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Для русского языка можно использовать модель sberbank-ai/rugpt3small_based_on_gpt2

In [16]:
model_name = 'sberbank-ai/rugpt3medium_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

KeyboardInterrupt: 

In [ ]:
# Создание пайплайна генерации
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

In [ ]:
prompt = "Заходит англичанин в бар"

In [ ]:
# Генерация анекдота
generated_joke = generator(
    prompt,  
    max_length=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    repetition_penalty=1.2,
    top_p=0.9,
    temperature=0.6,
    do_sample=True,
    top_k=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id  # Добавляем pad_token_id
)

print(generated_joke[0]['generated_text'])